In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
pip install  pytesseract


Note: you may need to restart the kernel to use updated packages.


In [5]:
!apt-get update
!apt-get install -y tesseract-ocr

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]                                
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]                
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease                                              
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]          
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,380 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [3,813 kB]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,767 kB]                       
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,698 kB]              
Get:11 http://security.ubuntu.com/ubuntu ja

In [6]:
!pip install sentence-transformers

In [ ]:
import os
import pandas as pd
from PIL import Image
import pytesseract
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torch.optim as optim
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics import accuracy_score, f1_score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

class AnxietyDataset(Dataset):
    def __init__(self, csv_file, image_folder, transform=None):
        self.image_folder = image_folder
        self.transform = transform
        
        data = pd.read_csv(csv_file)
        print("CSV columns:", data.columns)
        
        print("First few sample_ids from CSV:")
        print(data['sample_id'].head())
    
        data['filename'] = data['sample_id'].apply(lambda x: str(x) + ".jpg" if not str(x).endswith(".jpg") else x)
        
        data['label'] = pd.factorize(data['meme_anxiety_categories'])[0]
        print("Unique label mapping:", dict(enumerate(pd.factorize(data['meme_anxiety_categories'])[1])))
    
        data['exists'] = data['filename'].apply(lambda x: os.path.exists(os.path.join(self.image_folder, x)))
        print("Number of existing files found:", data['exists'].sum())
        
        self.data = data[data['exists']].reset_index(drop=True)
        print(f"Filtered dataset length: {len(self.data)} (only rows with existing images)")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        image_name = row['filename']
        label = row['label']
        image_path = os.path.join(self.image_folder, image_name)
        
        image = Image.open(image_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
    
        ocr_text = pytesseract.image_to_string(Image.open(image_path).convert("RGB"))
        return ocr_text, label

class FigurativeReasoningModule(nn.Module):
    def __init__(self):
        super(FigurativeReasoningModule, self).__init__()
    
    def forward(self, ocr_text):
        reasoning = "dummy figurative reasoning for: " + ocr_text
        return reasoning

class RAGModule(nn.Module):
    def __init__(self, model_name='paraphrase-MiniLM-L6-v2'):
        super(RAGModule, self).__init__()
        self.encoder = SentenceTransformer(model_name)

    def encode_text(self, text):
        embedding = self.encoder.encode(text, convert_to_tensor=True)
        return embedding

    def fuse_embeddings(self, ocr_text, reasoning):
        emb_ocr = self.encode_text(ocr_text)
        emb_reasoning = self.encode_text(reasoning)
        fused = torch.cat([emb_ocr, emb_reasoning], dim=-1)
        return fused.to(device)

    def retrieve_similar(self, query_embedding, database_embeddings, top_k=1):
        cos_scores = util.cos_sim(query_embedding, database_embeddings)
        top_results = torch.topk(cos_scores, k=top_k)
        return top_results

class M3HClassifier(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(M3HClassifier, self).__init__()
        self.fc = nn.Linear(input_dim, num_classes)
    
    def forward(self, fused_embedding):
        logits = self.fc(fused_embedding)
        return logits

class M3HModel(nn.Module):
    def __init__(self, rag_module, classifier):
        super(M3HModel, self).__init__()
        self.rag_module = rag_module
        self.classifier = classifier
    
    def forward(self, ocr_text, reasoning):
        fused_embedding = self.rag_module.fuse_embeddings(ocr_text, reasoning)
        if fused_embedding.dim() == 1:
            fused_embedding = fused_embedding.unsqueeze(0)
        logits = self.classifier(fused_embedding)
        return logits

if __name__ == "__main__":
    rag_module = RAGModule()
    figurative_module = FigurativeReasoningModule()
    fused_dim = 768  
    num_classes = 7  
    classifier = M3HClassifier(input_dim=fused_dim, num_classes=num_classes)
    model = M3HModel(rag_module, classifier)
    model.to(device) 
    
    optimizer = optim.Adam(model.parameters(), lr=5e-5)
    criterion = nn.CrossEntropyLoss()
    
    csv_train = '/kaggle/input/anxt-dataset/anxiety_train.csv'
    image_folder = '/kaggle/input/anxt-dataset/anxiety_train_image'
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])
    dataset = AnxietyDataset(csv_file=csv_train, image_folder=image_folder, transform=transform)
    dataloader = DataLoader(dataset, batch_size=4, shuffle=True)
    
    num_epochs = 10
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0.0
        for batch in dataloader:
            ocr_texts, labels = batch  
            
            reasoning_batch = [figurative_module(text) for text in ocr_texts]
            
            logits = model(ocr_texts, reasoning_batch)  
            
            labels = torch.tensor(labels, dtype=torch.long).to(device)
            
            loss = criterion(logits, labels)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
            print(f"Epoch {epoch+1} Batch loss: {loss.item()}")
        
        avg_loss = total_loss / len(dataloader)
        print(f"Epoch {epoch+1} Average loss: {avg_loss}")
    

Using device: cuda
CSV columns: Index(['sample_id', 'meme_anxiety_categories'], dtype='object')
First few sample_ids from CSV:
0     TR-318
1    TR-1732
2    TR-1417
3    TR-1582
4     TR-649
Name: sample_id, dtype: object
Unique label mapping: {0: 'Restlessness', 1: 'Lack of Worry Control', 2: 'Nervousness', 3: 'Impending Doom', 4: 'Excessive Worry', 5: 'Difficulty Relaxing', 6: 'Irritatbily'}
Number of existing files found: 2605
Filtered dataset length: 2605 (only rows with existing images)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

<ipython-input-9-a58de5a3210a>:175: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels, dtype=torch.long).to(device)


Epoch 1 Batch loss: 2.0387890338897705


...

Epoch 10 Batch loss: 1.9644296169281006


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Epoch 10 Batch loss: 1.6395361423492432
Epoch 10 Average loss: 1.5847314541325248


<ipython-input-9-a58de5a3210a>:175: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels, dtype=torch.long).to(device)


In [ ]:
# Evaluation
model.eval()
all_preds = []
all_labels = []
with torch.no_grad():
    for batch in dataloader:  
        ocr_texts, labels = batch
        reasoning_batch = [figurative_module(text) for text in ocr_texts]
        logits = model(ocr_texts, reasoning_batch)
        preds = torch.argmax(logits, dim=-1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(all_labels, all_preds)
macro_f1 = f1_score(all_labels, all_preds, average='macro')
weighted_f1 = f1_score(all_labels, all_preds, average='weighted')

print("Evaluation Metrics:")
print("Accuracy:", accuracy)
print("Macro-F1:", macro_f1)
print("Weighted-F1:", weighted_f1)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Evaluation Metrics:
Accuracy: 0.42687140115163147
Macro-F1: 0.3669940883213095
Weighted-F1: 0.40521547775758876


In [ ]:

# Evaluation on Test Set

test_csv = '/kaggle/input/anxt-dataset/anxiety_test.csv'
test_image_folder = '/kaggle/input/anxt-dataset/anxiety_test_image'

test_dataset = AnxietyDataset(csv_file=test_csv, image_folder=test_image_folder, transform=transform)
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=False)

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in test_dataloader:
        ocr_texts, labels = batch
        reasoning_batch = [figurative_module(text) for text in ocr_texts]
        logits = model(ocr_texts, reasoning_batch)
        preds = torch.argmax(logits, dim=-1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(torch.tensor(labels).cpu().numpy())
1r 
accuracy = accuracy_score(all_labels, all_preds)
macro_f1 = f1_score(all_labels, all_preds, average='macro')
weighted_f1 = f1_score(all_labels, all_preds, average='weighted')

print("Test Evaluation Metrics:")
print("Accuracy:", accuracy)
print("Macro-F1:", macro_f1)
print("Weighted-F1:", weighted_f1)


CSV columns: Index(['sample_id', 'meme_anxiety_categories'], dtype='object')
First few sample_ids from CSV:
0    TE-515
1    TE-203
2    TE-141
3    TE-415
4    TE-386
Name: sample_id, dtype: object
Unique label mapping: {0: 'Impending Doom', 1: 'Nervousness', 2: 'Restlessness', 3: 'Lack of Worry Control', 4: 'Irritatbily', 5: 'Difficulty Relaxing', 6: 'Excessive Worry'}
Number of existing files found: 651
Filtered dataset length: 651 (only rows with existing images)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

<ipython-input-11-eaa8b41bf794>:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  all_labels.extend(torch.tensor(labels).cpu().numpy())


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

<ipython-input-11-eaa8b41bf794>:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  all_labels.extend(torch.tensor(labels).cpu().numpy())


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Test Evaluation Metrics:
Accuracy: 0.18740399385560677
Macro-F1: 0.17047481499130018
Weighted-F1: 0.17780908953466878


<ipython-input-11-eaa8b41bf794>:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  all_labels.extend(torch.tensor(labels).cpu().numpy())
